## CWL Signup Processor

Use this notebook script to process a list of CoC player tags

In [1]:
import pandas as pd

# Read in data sheet
data = pd.read_csv('CWL Signup Form (Responses) - Form Responses 1.csv')
data = data.drop(['Timestamp', 'What is your Discord name?', 'Do you have any comments or feedback for us regarding CWL?'], axis=1)


# Clean up player tags, if needed
def clean_tag(tag):
    if '#' not in tag:
        tag =  '#' + tag
    tag =  tag.upper()
    tag = tag.replace('O', '0')
    return tag

tags = []
for key, val in data.iterrows():
    for item in val:
        if type(item) is str:
            tags.append(clean_tag(item))

In [2]:
import coc
import nest_asyncio
nest_asyncio.apply()

client = coc.login('nrcbot@gmail.com', 'PhantomBot1234', 
                   client=coc.Client, key_names='NRC Bot', key_count=1)

In [3]:
df = pd.DataFrame(columns=['Tag', 'Name', 'TH', 'BK', 'AQ', 'GW', 'RC'])

for tag in tags:
    try:
        player = await client.get_player(tag)
        name = player.name
        TH = player.town_hall
        if TH < 11:
            continue
        BK = player.get_hero('Barbarian King').level
        AQ = player.get_hero('Archer Queen').level
        if player.get_hero('Grand Warden', default_value=0) == 0: GW = 0
        else: GW = player.get_hero('Grand Warden', default_value=0).level
        if player.get_hero('Royal Champion', default_value=0) == 0: RC = 0
        else: RC = player.get_hero('Royal Champion', default_value=0).level
        df = df.append({'Tag':tag, 'Name':name, 'TH':TH, 'BK':BK, 'AQ':AQ, 'GW':GW, 'RC':RC}, ignore_index=True)
    except Exception:
        print(tag)

In [4]:
df = df.sort_values(['TH', 'BK', 'AQ', 'GW', 'RC', 'Name'], ascending=False)
df.to_excel('CWL.xlsx', index=False)